In [51]:
%pip install pandas
%pip install matplotlib
%pip install imbalanced-learn
%pip install numpy
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take som

In [52]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestNeighbors
from sklearn.utils import resample

In [53]:
df = pd.read_csv('good_customer.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1723 entries, 0 to 1722
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   month                1723 non-null   int64 
 1   credit_amount        1723 non-null   int64 
 2   credit_term          1723 non-null   int64 
 3   age                  1723 non-null   int64 
 4   sex                  1723 non-null   object
 5   education            1723 non-null   object
 6   product_type         1723 non-null   object
 7   having_children_flg  1723 non-null   int64 
 8   region               1723 non-null   int64 
 9   income               1723 non-null   int64 
 10  family_status        1723 non-null   object
 11  phone_operator       1723 non-null   int64 
 12  is_client            1723 non-null   int64 
 13  bad_client_target    1723 non-null   int64 
dtypes: int64(10), object(4)
memory usage: 188.6+ KB


In [54]:
df.head(10)

,month,credit_amount,credit_term,age,sex,education,product_type,having_children_flg,region,income,family_status,phone_operator,is_client,bad_client_target
0,1,7000,12,39,male,Secondary special education,Cell phones,0,2,21000,Another,0,0,0
1,1,19000,6,20,male,Secondary special education,Household appliances,1,2,17000,Another,3,1,0
2,1,29000,12,23,female,Secondary special education,Household appliances,0,2,31000,Another,2,0,0
3,1,10000,12,30,male,Secondary special education,Cell phones,1,2,31000,Unmarried,3,1,0
4,1,14500,12,25,female,Higher education,Cell phones,0,2,26000,Married,0,1,0
5,1,32500,24,47,female,Secondary special education,Furniture,0,2,26000,Married,0,1,0
6,1,8000,3,23,male,Higher education,Computers,0,2,21000,Another,0,1,0
7,1,20000,10,25,female,Higher education,Household appliances,0,0,33000,Married,2,1,0
8,1,26000,6,21,female,Secondary special education,Cell phones,0,0,31000,Another,2,1,0
9,1,15000,24,25,female,Secondary special education,Household appliances,1,2,26000,Another,3,0,0


In [55]:
rows_with_nan = df.isna().any(axis=1)

# Display rows that contain at least one NaN value
nan_rows = df[rows_with_nan]

nan_rows

,month,credit_amount,credit_term,age,sex,education,product_type,having_children_flg,region,income,family_status,phone_operator,is_client,bad_client_target


In [56]:
df.describe()

,month,credit_amount,credit_term,age,having_children_flg,region,income,phone_operator,is_client,bad_client_target
count,1723.000000,1723.000000,1723.000000,1723.000000,1723.000000,1723.000000,1723.000000,1723.000000,1723.000000,1723.000000
mean,6.708067,29264.654672,11.546721,35.911782,0.428323,1.681370,32652.350551,1.125363,0.604759,0.113755
std,3.538420,27926.778301,6.548354,13.120203,0.494979,0.704256,20913.193158,1.015822,0.489044,0.317606
min,1.000000,5000.000000,3.000000,18.000000,0.000000,0.000000,1000.000000,0.000000,0.000000,0.000000
25%,3.000000,13000.000000,6.000000,26.000000,0.000000,2.000000,21000.000000,0.000000,0.000000,0.000000
50%,7.000000,21500.000000,12.000000,32.000000,0.000000,2.000000,27000.000000,1.000000,1.000000,0.000000
75%,10.000000,34000.000000,12.000000,44.000000,1.000000,2.000000,38000.000000,2.000000,1.000000,0.000000
max,12.000000,301000.000000,36.000000,90.000000,1.000000,2.000000,401000.000000,4.000000,1.000000,1.000000


In [57]:
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE


label_encoder = LabelEncoder()
df["sex"] = label_encoder.fit_transform(df["sex"])
df["education"] = label_encoder.fit_transform(df["education"])
df["product_type"] = label_encoder.fit_transform(df["product_type"])
df["family_status"] = label_encoder.fit_transform(df["family_status"])


X = df.drop("bad_client_target", axis=1)
y = df["bad_client_target"]

In [58]:
class_counts = df["bad_client_target"].value_counts()
minority_class = class_counts.idxmin()
majority_class = class_counts.idxmax()

In [59]:
print("Index for minority class:", minority_class)
print("Index for majority class:", majority_class)

Index for minority class: 1
Index for majority class: 0


In [60]:
# Define features
features = df.select_dtypes(include=np.number).drop("bad_client_target", axis=1)
print("Features:", list(features.columns))

Features: ['month', 'credit_amount', 'credit_term', 'age', 'sex', 'education', 'product_type', 'having_children_flg', 'region', 'income', 'family_status', 'phone_operator', 'is_client']


In [61]:
# Nearest Neighbors for proportion calculation
neighbors = NearestNeighbors(n_neighbors=5)
neighbors.fit(features)
distances, indices = neighbors.kneighbors(features)
flat_indices = indices.flatten()
neighbor_classes_flat = df.iloc[flat_indices]["bad_client_target"].values.reshape(
    indices.shape
)
proportions = np.mean(neighbor_classes_flat == minority_class, axis=1)

In [62]:
# Define thresholds and partition the data
boundary_threshold = 0.3
noise_threshold = 0.1
boundary_samples = df[
    (proportions >= boundary_threshold) & (proportions <= (1 - boundary_threshold))
]
noise_samples = df[proportions < noise_threshold]
data_without_noise = df.drop(noise_samples.index)

In [63]:
# Cluster minority samples
minority_samples = data_without_noise[
    data_without_noise["bad_client_target"] == minority_class
]
cluster = AgglomerativeClustering(n_clusters=2, affinity="euclidean", linkage="ward")
minority_samples["cluster_label"] = cluster.fit_predict(
    minority_samples[features.columns]
)
df.loc[minority_samples.index, "cluster_label"] = minority_samples["cluster_label"]

/home/stavre/miniconda3/envs/pyenv_rcs/lib/python3.8/site-packages/sklearn/cluster/_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
/tmp/ipykernel_422681/3503392604.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minority_samples["cluster_label"] = cluster.fit_predict(


In [64]:
# Recalculate boundary_minority_samples with cluster labels
boundary_minority_samples = boundary_samples[
    boundary_samples["bad_client_target"] == minority_class
]
boundary_minority_samples = boundary_minority_samples.merge(
    df[["cluster_label"]], left_index=True, right_index=True, how="left"
)

In [65]:
# Weights for boundary minority samples
_, boundary_indices = neighbors.kneighbors(boundary_minority_samples[features.columns])
boundary_neighbor_classes = df.iloc[boundary_indices.flatten()][
    "bad_client_target"
].values.reshape(boundary_indices.shape)
boundary_weights = np.mean(boundary_neighbor_classes == majority_class, axis=1)
boundary_weights_normalized = boundary_weights / np.sum(boundary_weights)

In [66]:
import random
# Generate synthetic samples
synthetic_samples = []
for index, sample in boundary_minority_samples.iterrows():
    if index in boundary_weights_map:
        cluster_label = sample["cluster_label"]
        sample_weight = boundary_weights_map[index]

        # Use a probabilistic approach for deciding the number of synthetic samples
        probability = sample_weight * 100  # adjust scale as needed
        num_synthetic_samples = (
            random.randint(0, round(probability)) if probability > 0 else 0
        )

        same_cluster_samples = minority_samples[
            minority_samples["cluster_label"] == cluster_label
        ]
        if not same_cluster_samples.empty and num_synthetic_samples > 0:
            synthetic = resample(
                same_cluster_samples,
                n_samples=num_synthetic_samples,
                random_state=42,
                replace=True,
            )
            synthetic_samples.extend(synthetic.values)

# Convert synthetic samples to DataFrame
synthetic_samples_df = pd.DataFrame(
    synthetic_samples, columns=boundary_minority_samples.columns
)

# Output the number of generated synthetic samples
print("Number of Synthetic Samples Generated:", len(synthetic_samples_df))

Number of Synthetic Samples Generated: 27


In [67]:
synthetic_samples_df.head(10)

,month,credit_amount,credit_term,age,sex,education,product_type,having_children_flg,region,income,family_status,phone_operator,is_client,bad_client_target,cluster_label
0,8,13000,12,23,1,5,13,1,2,16000,1,1,1,1,0
1,8,13000,12,23,1,5,13,1,2,16000,1,1,1,1,0
2,8,13000,12,23,1,5,13,1,2,16000,1,1,1,1,0
3,8,13000,12,23,1,5,13,1,2,16000,1,1,1,1,0
4,8,13000,12,23,1,5,13,1,2,16000,1,1,1,1,0
5,8,13000,12,23,1,5,13,1,2,16000,1,1,1,1,0
6,8,13000,12,23,1,5,13,1,2,16000,1,1,1,1,0
7,8,13000,12,23,1,5,13,1,2,16000,1,1,1,1,0
8,8,13000,12,23,1,5,13,1,2,16000,1,1,1,1,0
9,8,13000,12,23,1,5,13,1,2,16000,1,1,1,1,0
